In [17]:
!pip install ultralytics
!pip install ultralytics pillow opencv-python

In [28]:
import os
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO

In [29]:
# === Load Pretrained YOLOv8 Model ===
yolo_model = YOLO('yolov8n.pt')  # You can switch to yolov8s.pt, yolov8m.pt, etc.

In [30]:
# === Load Any Image Format ===
def load_image_any_format(img_path):
    img_pil = Image.open(img_path)
    if img_pil.format == 'GIF':
        img_pil.seek(0)
    img_pil = img_pil.convert('RGB')
    img_np = np.array(img_pil)
    return cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

In [31]:
def detect_objects(img_path, expected_objects=None, save_annotated=False, output_dir="./detections"):
    image = load_image_any_format(img_path)
    results = yolo_model(image)

    detected_objects = []
    match_found = False

    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = yolo_model.names[cls_id]
            detected_objects.append((label, conf))

            if expected_objects is None or label.lower() in expected_objects:
                match_found = True
                print(f"[MATCHED] {label} ({conf:.2f})")

    if save_annotated and match_found:
        os.makedirs(output_dir, exist_ok=True)
        save_path = os.path.join(output_dir, os.path.basename(img_path))
        results[0].save(filename=save_path)
        print(f"✅ Saved annotated image to {save_path}")

    return detected_objects if match_found else []


In [32]:
# === Batch Detection With Filtering ===
def detect_in_folder(folder_path, output_folder="./detections", expected_objects=None, save_annotated=True):
    supported_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif")
    expected_objects = [obj.lower() for obj in expected_objects] if expected_objects else None

    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(supported_exts):
            img_path = os.path.join(folder_path, img_name)
            print(f"\n📂 Processing {img_name}")
            detected = detect_objects(img_path, expected_objects=expected_objects, save_annotated=save_annotated, output_dir=output_folder)
            if not detected:
                print("No expected objects found.")

In [ ]:
# === Main Runner ===
image_folder = r"C:\Users\scott\OneDrive\Documents\Protocols\Python\PyCharm\workspaces\pycharm_windows\AI_Projects\ai_cyberforensics\datasets\images\objects\raw"
output_folder = r"C:\Users\scott\OneDrive\Documents\Protocols\Python\PyCharm\workspaces\pycharm_windows\AI_Projects\ai_cyberforensics\datasets\images\objects\detections"
expected_objects = ["knife", "gun"]  # You can modify this

detect_in_folder(image_folder, output_folder=output_folder, expected_objects=expected_objects, save_annotated=True)



📂 Processing --------_------_jpg.rf.08c002808911e42dcb831337e2eed710.jpg

0: 640x640 1 vase, 186.9ms
Speed: 5.9ms preprocess, 186.9ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
No expected objects found.

📂 Processing --------_------_jpg.rf.0c6fff42bd233e5fb0e2ef448b4e4db1.jpg

0: 640x640 (no detections), 116.1ms
Speed: 5.6ms preprocess, 116.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
No expected objects found.

📂 Processing --------_------_jpg.rf.148da60814a734f1e592d3e89b784c47.jpg

0: 640x640 (no detections), 116.7ms
Speed: 4.2ms preprocess, 116.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
No expected objects found.

📂 Processing --------_------_jpg.rf.2b5bd69b5834c97c3cf5951759c2b640.jpg

0: 640x640 (no detections), 120.2ms
Speed: 4.0ms preprocess, 120.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
No expected objects found.

📂 Processing --------_------_jpg.rf.2f14618aacad49b58b36d109b02f

W

In [3]:
# object_detection.py

import os
from PIL import Image
import numpy as np
import cv2
import shutil
from ultralytics import YOLO

# === Load YOLOv8 Pretrained Model ===
yolo_model = YOLO('yolov8x.pt')  # You can upgrade to yolov8s/m/x if needed

# === Load Any Image Format (GIF-safe) ===
def load_image_any_format(img_path):
    img_pil = Image.open(img_path)
    if img_pil.format == 'GIF':
        img_pil.seek(0)
    img_pil = img_pil.convert('RGB')
    img_np = np.array(img_pil)
    return cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

# === Run Detection and Copy Original on Match ===
def detect_objects_and_copy(img_path, expected_objects, output_dir):
    image = load_image_any_format(img_path)
    results = yolo_model(image)

    expected_objects = [obj.lower() for obj in expected_objects]
    match_found = False

    for result in results:
        for box in result.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            label = yolo_model.names[cls_id].lower()

            if label in expected_objects:
                print(f"[MATCHED] {label} ({conf:.2f})")
                match_found = True

    if match_found:
        os.makedirs(output_dir, exist_ok=True)
        dest_path = os.path.join(output_dir, os.path.basename(img_path))
        shutil.copy2(img_path, dest_path)
        print(f"✅ Copied original image to {dest_path}")

# === Batch Detection Runner ===
def detect_in_folder(folder_path, output_folder="./matches", expected_objects=None):
    supported_exts = (".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif")
    if expected_objects is None or len(expected_objects) == 0:
        print("⚠️ You must specify a list of expected objects (e.g., ['knife', 'gun'])")
        return

    for img_name in os.listdir(folder_path):
        if img_name.lower().endswith(supported_exts):
            img_path = os.path.join(folder_path, img_name)
            print(f"\n📂 Processing {img_name}")
            detect_objects_and_copy(img_path, expected_objects, output_folder)

# === Main Runner ===

image_folder = r"C:\Users\scott\OneDrive\Documents\Protocols\Python\PyCharm\workspaces\pycharm_windows\AI_Projects\ai_cyberforensics\datasets\images\objects\raw"
output_folder = r"C:\Users\scott\OneDrive\Documents\Protocols\Python\PyCharm\workspaces\pycharm_windows\AI_Projects\ai_cyberforensics\datasets\images\objects\detections"
expected_objects = ["knife"]  # You can modify this

detect_in_folder(image_folder, output_folder=output_folder, expected_objects=expected_objects)


100%|██████████| 131M/131M [00:08<00:00, 15.4MB/s] 



📂 Processing 445_jpg.rf.2e04379013684f454abbc00564910fcc.jpg

0: 640x640 1 scissors, 1099.7ms
Speed: 6.3ms preprocess, 1099.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

📂 Processing ABbframe00154_jpg.rf.d56723a3de6d775966551d0d0e8add67.jpg

0: 640x640 1 person, 1 tie, 1 potted plant, 1 cell phone, 1122.1ms
Speed: 4.2ms preprocess, 1122.1ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

📂 Processing ABbframe00160_jpg.rf.23c17f2cfeacd7873e38995e4832e0c1.jpg

0: 640x640 1 person, 1 cell phone, 1071.2ms
Speed: 4.0ms preprocess, 1071.2ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

📂 Processing ABbframe00166_jpg.rf.d0f4052c7acfd9a298f657d3c56279e3.jpg

0: 640x640 1 person, 1078.4ms
Speed: 4.5ms preprocess, 1078.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

📂 Processing ABbframe00169_jpg.rf.0a602b61133774dd78c98b0031ac834a.jpg

0: 640x640 1 person, 1058.0ms
Speed: 4.1ms preprocess, 1058.0ms inference, 1